In [1]:
:opt no-lint

# Simply-typed $\lambda$-calculus (STLC)

단순타입 람다계산법(Simply-typed $\lambda$-calculus 또는 줄여서 STLC)

$\tau \in \textit{Ty}
      ::= \tau_1 \to \tau_2
    ~\mid~ \textit{Int}
$

$x\in \textit{Nm} \qquad n\in \textit{Int}$

$e \in \textit{Expr}
   ::=~ x ~\mid~ \lambda x:\tau.e ~\mid~ e_1~e_2 ~\mid~ \\\qquad\qquad
 ~\mid~ n ~\mid~ e_1 + e_2 ~\mid~ \texttt{if}~e~\texttt{then}~e_1~\texttt{else}~e_0
$

이렇게 타입있는 람다계산법 문법에서 함수정의식에 나타나는 매개변수 옆에
그 매개변수가 가져야 할 타입을 적어 주는 형태가 Church-style이다.

즉 위에서 나타난 문법 정의는 Church-style STLC이다.

In [2]:
data Ty = Ty :-> Ty | INT  deriving (Eq,Show)
infixr 9 :->

data Expr = Var Nm  | Lam (Nm,Ty) Expr | App Expr Expr
          | Val Int | Add Expr Expr | If Expr Expr Expr
        deriving Show

type Nm = String

In [3]:
ppTy (t1 :-> t2) = ppty t1 ++ "->" ++ ppTy t2
ppTy INT = "int"

ppty t@(_ :-> _) = paren (ppTy t)
ppty t           = ppTy t

-- 람다식을 보기좋게 문자열로 변환해주는 함수
ppTm (Var x) = x
ppTm (Lam (x,t) e) = "\\"++x++":"++ppty t++" -> " ++ ppTm e
ppTm (App e1 e2) = pp1 e1 ++ " " ++ pp2 e2
ppTm (Val n) = show n
ppTm (Add e1 e2) = ppp e1 ++ " + " ++ ppp e2
ppTm (If e e1 e0) = "if "++pp2 e++" then "++pp2 e1++" else "++pp2 e0 

pp1 e@(Lam{}) = paren (ppTm e)
pp1 e@(Add{}) = paren (ppTm e)
pp1 e@(If{})  = paren (ppTm e)
pp1 e         = ppTm e

pp2 e@(Var{}) = ppTm e
pp2 e@(Val{}) = ppTm e
pp2 e         = paren (ppTm e)

ppp e@(Var{}) = ppTm e
ppp e@(Val{}) = ppTm e
ppp e@(Add{}) = ppTm e
ppp e@(App{}) = ppTm e
ppp e         = paren (ppTm e)

paren s = "(" ++ s ++ ")"
brack s = "[" ++ s ++ "]"
latex s = "$" ++ s ++ "$"

texTy (t1 :-> t2) = texty t1 ++ "\\!\\to\\!" ++ texTy t2
texTy INT = "\\texttt{int}"

texty t@(_ :-> _) = paren (texTy t)
texty t           = texTy t

-- 람다식을 보기좋게 TeX 코드로 변환해주는 함수
texTm (Var x) = x
texTm (Lam (x,t) e) = "\\lambda " ++ x ++"\\!:\\!"++texty t++ "." ++ texTm e
texTm (App e1 e2) = tex1 e1 ++ "~" ++ tex2 e2
texTm (Val n) = show n
texTm (Add e1 e2) = texp e1 ++ "+" ++ texp e2
texTm (If e e1 e0) = "\\texttt{if}~"++tex2 e++"~\\texttt{then}~"++tex2 e1++"~\\texttt{else}~"++tex2 e0

tex1 e@(Lam{}) = paren (texTm e)
tex1 e@(Add{}) = paren (texTm e)
tex1 e@(If{}) = paren (texTm e)
tex1 t         = texTm t

tex2 s@(Var{}) = texTm s
tex2 s@(Val{}) = texTm s
tex2 s         = paren (texTm s)

texp s@(Var{}) = texTm s
texp s@(Val{}) = texTm s
texp s@(Add{}) = texTm s
texp s@(App{}) = texTm s
texp s         = paren (texTm s)

In [4]:
idI = Lam ("x",INT) (Var "x")
idI2I = Lam ("x",INT:->INT) (Var "x")
idII2I = Lam ("x",INT:->INT:->INT) (Var "x")
addTm = Lam ("x",INT) $ Lam ("y",INT) $ Var "x" `Add` Var "y" 

putStr $ ppTm idI
putStr $ ppTm idI2I
putStr $ ppTm idII2I
putStr $ ppTm addTm

\x:int -> x

\x:(int->int) -> x

\x:(int->int->int) -> x

\x:int -> \y:int -> x + y

In [5]:
import IHaskell.Display

html . latex $ texTm idI
html . latex $ texTm idI2I
html . latex $ texTm idII2I
html . latex $ texTm addTm
html . latex $ texTm (App idII2I addTm)

$\lambda x\!:\!\texttt{int}.x$

$\lambda x\!:\!(\texttt{int}\!\to\!\texttt{int}).x$

$\lambda x\!:\!(\texttt{int}\!\to\!\texttt{int}\!\to\!\texttt{int}).x$

$\lambda x\!:\!\texttt{int}.\lambda y\!:\!\texttt{int}.x+y$

$(\lambda x\!:\!(\texttt{int}\!\to\!\texttt{int}\!\to\!\texttt{int}).x)~(\lambda x\!:\!\texttt{int}.\lambda y\!:\!\texttt{int}.x+y)$

----
## Church-style STLC 타입 규칙

$\displaystyle
\textit{TEnv} ~=~ \textit{Nm}\xrightarrow{\textrm{fin}}\textit{Ty}\\
\Gamma \in \textit{TEnv} ::= x_1:\tau_1,\;\ldots\;,\,x_n:\tau_n
$

$\displaystyle
\frac{\tau = \Gamma(x)}{\Gamma \vdash x : \tau}
\qquad
\frac{ x:\tau,\Gamma \vdash e : \tau_2
    }{ \Gamma \vdash (\lambda x:\tau.\,e) : \tau\to\tau_2 }
\\~\\ \displaystyle
\frac{ \Gamma \vdash e_1 : \tau_2\to\tau \quad \Gamma \vdash e_2 : \tau_2 
    }{ \Gamma \vdash (e_1~e_2) : \tau }
\\~\\ \displaystyle
\frac{
    }{ \Gamma \vdash n : \texttt{int} }
\qquad
\frac{ \Gamma \vdash e_1 : \texttt{int} \quad \Gamma \vdash e_2 : \texttt{int}
    }{ \Gamma \vdash e_1 + e_2 : \texttt{int} }
\\~\\ \displaystyle
\frac{ \Gamma \vdash e : \texttt{int} \quad
       \Gamma \vdash e_1 : \tau \quad \Gamma \vdash e_0 : \tau
    }{ \Gamma \vdash \texttt{if}\,~e~\,\texttt{then}\,~e_1~\texttt{else}\,~e_0 : \tau}
$

In [14]:
type TEnv = [ (Nm, Ty) ]

tyinf :: TEnv -> Expr -> Ty
tyinf tenv (Var x) = lookup' x tenv
tyinf tenv (Lam (x,t) e) = t :-> t2
  where
    t2 = tyinf ((x,t):tenv) e
tyinf tenv (App e1 e2) =
  case t1 of
    t2' :-> t  | t2==t2'   -> t
               | otherwise -> error $ show e1 ++" : " ++ ppty t1
                                   ++ " cannot be applied to "
                                   ++ show e2 ++" : " ++ ppty t2
    _ -> error $ show e1 ++ " : " ++ ppty t1
              ++ " is not a function type"
  where
    t1 = tyinf tenv e1
    t2 = tyinf tenv e2
tyinf tenv (Val _) = INT
tyinf tenv (Add e1 e2) =
  case (t1, t2) of
    (INT, INT) -> INT
    (INT, _  ) -> error $ show e2 ++" : " ++ ppty t2 ++ " is not INT type"
    _          -> error $ show e1 ++" : " ++ ppty t1 ++ " is not INT type"
  where
    t1 = tyinf tenv e1
    t2 = tyinf tenv e2
tyinf tenv (If e e1 e0) =
  case t of
    INT | t1 == t0  -> t1
        | otherwise -> error $ show e1 ++" : " ++ ppty t1 ++ " and "
                            ++ show e0 ++" : " ++ ppty t0 ++ " are different type"
    _ -> error $ show e ++" : " ++ ppty t ++ " is not INT type"
  where
    t = tyinf tenv e
    t1 = tyinf tenv e1
    t0 = tyinf tenv e0

lookup' x env = case lookup x env of
                  Nothing -> error (x ++ " not defined")
                  Just v  -> v

In [15]:
add1 = Lam ("x",INT) $ Add (Val 1) (Var "x")

html . latex $ texTm add1
html . latex . texTy $ tyinf [] add1

$\lambda x\!:\!\texttt{int}.1+x$

$\texttt{int}\!\to\!\texttt{int}$

In [22]:
html . latex $ texTm (App add1 (Val 3))
html . latex . texTy $ tyinf [] (App add1 (Val 3))

$(\lambda x\!:\!\texttt{int}.1+x)~3$

$\texttt{int}$

In [23]:
html . latex $ texTm (App (Val 3) add1)
html . latex . texTy $ tyinf [] (App (Val 3) add1)

$3~(\lambda x\!:\!\texttt{int}.1+x)$

: 

In [24]:
html . latex $ texTm (App add1 idI)
html . latex . texTy $ tyinf [] (App add1 idI)

$(\lambda x\!:\!\texttt{int}.1+x)~(\lambda x\!:\!\texttt{int}.x)$

: 

In [25]:
html . latex $ texTm (App idI add1)
html . latex . texTy $ tyinf [] (App idI add1)

$(\lambda x\!:\!\texttt{int}.x)~(\lambda x\!:\!\texttt{int}.1+x)$

: 

In [26]:
html . latex $ texTm (App idI2I add1)
html . latex . texTy $ tyinf [] (App idI2I add1)

$(\lambda x\!:\!(\texttt{int}\!\to\!\texttt{int}).x)~(\lambda x\!:\!\texttt{int}.1+x)$

$\texttt{int}\!\to\!\texttt{int}$